In [14]:
import sqlite3 as sql
import math
import catalog_utils as cu

#run_phase1('gtw49', '/home/kecai/w49/w49.db')

In [15]:
#UPDATE Phase field for sources that have m5_8 and m8_0 detections and whose
#photometric uncertainities < 0.2 mag in all four IRAC bands
#con is a connection object
#t is the name of the table from which we draw records
def init(con, source):
    cur = con.cursor()
    target = source + '_p1'
    cur.execute('''CREATE TABLE ''' + target + ''' AS 
                   SELECT desig, m3_6, m4_5, m5_8, m8_0, 
                                dm3_6, dm4_5, dm5_8, dm8_0, 
                                c3645, c3658, c3680, c4558, c4580, c5880,
                                Type FROM ''' + source + 
                ''' WHERE (m5_8 IS NOT NULL) AND 
                          (m8_0 IS NOT NULL) AND 
                          dm3_6 < ? AND 
                          dm4_5 < ? AND 
                          dm5_8 < ? AND 
                          dm8_0 < ?''', [0.2]*4)
    return target

In [16]:
#apply PAH conditions to extract PAH (Type = 19), update n_Type in t_p1
def extract_19(con, source):
    cur = con.cursor()
    #CREATE TABLE containing PAH galaxies
    target = source + '_19'
    cur.execute('''CREATE TABLE '''+ target + ''' AS 
                   SELECT desig FROM ''' + source + ''' WHERE Type IS NULL AND
                   ((c4558  < (1.05/1.2) * (c5880 - 1)) AND
                    (c4558 < 1.05) AND
                    (c5880 > 1) AND
                    (m4_5 > 11.5))
                    OR
                   ((c3658 < (1.5/2) * (c4580 -1)) AND
                    (c3658 < 1.5) AND
                    (c4580 > 1) AND
                    (m4_5 > 11.5))''')
    return target

In [17]:
def extract_29(con, source):
    cur = con.cursor()
    #CREATE TABLE containing AGN
    target = source + '_29'
    cur.execute('''CREATE TABLE '''+ target + ''' AS 
                   SELECT desig FROM ''' + source + ''' WHERE Type IS NULL AND
                  (c4580 > 0.5 AND m4_5 > 13.5 +(c4580 - 2.3)/0.4 AND m4_5 > 13.5) 
                  AND
                  (m4_5 > 14 + (c4580 - 0.5) OR m4_5 > 14.5 - (c4580 - 1.2)/0.3 OR m4_5 > 14.5)''')
    return target

In [18]:
def extract_9(con, source):
    cur = con.cursor()
    #CREATE TABLE containing shock gas emission 
    target = source + '_9'
    cur.execute('''CREATE TABLE '''+ target + ''' AS 
                   SELECT desig FROM '''+ source + ''' WHERE Type IS NULL AND 
                   (c3645 > (1.2/0.55)*(c4558 - 0.3)+0.8) AND
                   (c4558 <= 0.85) AND
                   (c3645 > 1.05)''')
    return target

In [19]:
def extract_20(con, source, applysigma=False):
    #CREATE TABLE containing pah contaminated apertures
    cur = con.cursor()
    target = source + '_20'
    if applysigma == False:
        cur.execute('''CREATE TABLE '''+ target +''' AS
                   SELECT desig FROM '''+ source +''' WHERE Type IS NULL AND
                   c3645 <= 1.4 * (c4558 - 0.7) + 0.15 AND
                   c3645 <= 1.65''')
    else:
        cur.execute('''CREATE TABLE '''+ target +''' AS
           SELECT desig FROM '''+ source +''' WHERE Type IS NULL AND
           c3645 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) <= 
              1.4 * (c4558 + SQRT(SQUARE(dm4_5) + SQUARE(dm5_8)) - 0.7) + 0.15 AND
           c3645 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) <= 1.65''')
    return target

In [20]:
def extract_1(con, source):
    cur = con.cursor()
    target = source + '_1'
    cur.execute('''CREATE TABLE '''+ target +''' AS 
                   SELECT desig FROM '''+ source +''' WHERE Type IS NULL AND
                   c4558 > 0.7 AND
                   c3645 > 0.7''')
    return target

In [21]:
def extract_2(con, source, applysigma=False):
    cur = con.cursor()
    target = source + '_2'
    if applysigma == False:
        cur.execute('''CREATE TABLE '''+ target +''' AS 
                       SELECT desig FROM '''+ source +''' WHERE Type IS NULL AND
                       c4580 > 0.5 AND
                       c3658 > 0.35 AND
                       c3658 <= (0.14/0.04)*(c4580 - 0.5) + 0.5 AND
                       c3645 > 0.15''')
    else:
        cur.execute('''CREATE TABLE '''+ target +''' AS 
                       SELECT desig FROM '''+ source +''' WHERE Type IS NULL AND
                       c4580 - SQRT(SQUARE(dm4_5) + SQUARE(dm8_0)) > 0.5 AND
                       c3658 - SQRT(SQUARE(dm3_6) + SQUARE(dm5_8)) > 0.35 AND
                       c3658 + SQRT(SQUARE(dm3_6) + SQUARE(dm5_8)) 
                         <= (0.14/0.04)*(c4580 - SQRT(SQUARE(dm4_5) + SQUARE(dm8_0))-0.5) + 0.5 AND
                       c3645 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) > 0.15''')
    return target

In [22]:
#requires two string arguments (put quotes around arguments!)
#master: name of the master table where all data is stored
#constr: connection string to the database
def run_phase1(master, constr, applysigma=False):
    con = sql.connect(constr)
    
    #add math udf to sqlite
    cu.add_math(con)
    #initialize and create a working copy for phase1 containing desig, four IRAC channel mags and Type
    wc = init(con, master) 
    cu.update_phase(con, master, wc, 1)
    
    #extract and update PAH in the working copy
    c19 = extract_19(con, wc) 
    cu.update_type(con, wc, c19, 19)
    cu.update_type(con, master, c19, 19)
    
    #extract and update AGN
    c29 = extract_29(con, wc)
    cu.update_type(con, wc, c29, 29)
    cu.update_type(con, master, c29, 29)
    
    c9 = extract_9(con, wc)
    cu.update_type(con, wc, c9, 9)
    cu.update_type(con, master, c9, 9)
    
    c20 = extract_20(con, wc, applysigma)
    cu.update_type(con, wc, c20, 20)
    cu.update_type(con, master, c20, 20)
    
    c1 = extract_1(con, wc)
    cu.update_type(con, wc, c1, 1)
    cu.update_type(con, master, c1, 1)
    
    c2 = extract_2(con, wc, applysigma)
    cu.update_type(con, wc, c2, 2)
    cu.update_type(con, master, c2, 2)
    
    return 0

In [23]:
run_phase1('gtw49', '/home/kecai/w49/w49test.db', 0)

0